In [1]:
from datasets import load_multitask_data
import bert
from config import PretrainedConfig
import torch
from datasets import SentencePairDataset
from tokenizer import BertTokenizer


In [2]:
#This library gives a better idea of how to compute the hidden states
# https://github.com/codertimo
# https://pypi.org/project/bert-pytorch/

In [3]:
#Install with pip install bert-pytorch
import bert_pytorch.model.embedding.bert as bert_module

In [4]:
# Load project's datasets
sentiment_data, num_labels, paraphrased_data, similarity_data = load_multitask_data("data/ids-sst-train.csv", "data/quora-train.csv", 
                    "data/sts-train.csv")
#Show example 
sentiment_data[0]

Loaded 8544 train examples from data/ids-sst-train.csv
Loaded 141498 train examples from data/quora-train.csv
Loaded 6040 train examples from data/sts-train.csv


("the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .",
 3,
 '32a4f146782cbde1b7fa65799')

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# Get initialization parameters to validate the methods we create
class BertConfig(PretrainedConfig):
  model_type = "bert"

  def __init__(
    self,
    vocab_size=30522,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    type_vocab_size=2,
    initializer_range=0.02,
    layer_norm_eps=1e-12,
    pad_token_id=0,
    gradient_checkpointing=False,
    position_embedding_type="absolute",
    use_cache=True,
    name_or_path = "checkpoint",
    **kwargs
  ):
    super().__init__(pad_token_id=pad_token_id, **kwargs)

    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.hidden_act = hidden_act
    self.intermediate_size = intermediate_size
    self.hidden_dropout_prob = hidden_dropout_prob
    self.attention_probs_dropout_prob = attention_probs_dropout_prob
    self.max_position_embeddings = max_position_embeddings
    self.type_vocab_size = type_vocab_size
    self.initializer_range = initializer_range
    self.layer_norm_eps = layer_norm_eps
    self.gradient_checkpointing = gradient_checkpointing
    self.position_embedding_type = position_embedding_type
    self.use_cache = use_cache
    self.name_or_path = name_or_path

In [10]:
config = BertConfig()

In [98]:
import importlib
importlib.reload(importlib.import_module("bert"))

bert_mod = bert.BertModel(config)

In [99]:
# Tokenizing and passing data

In [111]:
padded_length = 15

dummy_data = ["the rock is destined to be the 21st century 's new `` conan "]
dummy_data = dummy_data*128
#Still have not included CLS token
tokenized_dummy_data= []
for sequence in dummy_data:
    x = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sequence))
    x = x + [0] * (padded_length - len(x)) 
    tokenized_dummy_data.append(x)


In [112]:
dummy_tensor =  torch.LongTensor(tokenized_dummy_data)
dummy_tensor_attention_mask = (dummy_tensor != 0).to(torch.int)

In [113]:
hidden_states = bert_mod.embed(dummy_tensor)

In [115]:
hidden_states.shape

torch.Size([128, 15, 768])

In [114]:
bert_mod.encode(hidden_states,dummy_tensor_attention_mask)

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor